In [5]:
import plotly
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
import os
import volume as vol

folder_path = "/media/mfaria/Ganesha/20171219_backup_home_catec/Margarida/20190100_exploration/sim_data/valid"
#folder_path = "/home/mfaria/Flying_Octomap_code/src/data"
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(folder_path):
    for file in f:
        if "frontiers.csv" in file:
            files.append(os.path.join(r, file))
            

layout = go.Layout(
    title='Occupied Volume Sampled',
    titlefont=dict(
        family='Courier New, monospace',
        size=24,
    ),
    xaxis=dict(
        title='Time in minutes',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        dtick=10
    ),
    yaxis=dict(
        title='Explored volume (m )',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

In [6]:
def volumeDf(file_path, label):
    df = pd.read_csv(file_path)
    trace_free_volume = go.Scatter(
        x=df.time_ellapsed_millis/1000/60,
        y=df.occupied,
        name='Run ' + str(label),
        mode='lines'
    )
    return trace_free_volume

In [7]:
def maxOccupiedVolume(file_path):
    df = pd.read_csv(file_path)
    return max(df.occupied)

def maxTime(file_path):
    df = pd.read_csv(file_path)
    return df.time_ellapsed_millis.max()

millis = max([maxTime(f) for f in files]) ;
max_time = millis.max()

maximum_occupied_volume = max([maxOccupiedVolume(f) for f in files]) 
trace_max_explored_volume = go.Scatter(
    x=[0, max_time/1000/60],
    y=[maximum_occupied_volume, maximum_occupied_volume],
    name='Maximum explored volume',
    mode='lines',
)

In [8]:
traces = []
label = 1
for f in files:
    traces.append(volumeDf(f, label))
    label = label +1
traces.append(trace_max_explored_volume)
fig=dict(data=traces, layout=layout)
#plotly.offline.plot(fig, filename='./volume_exploration_vs_time.html', image='png')
plotly.offline.iplot(fig)
pio.write_image(fig, '/media/mfaria/Ganesha/20171219_backup_home_catec/Margarida/20190100_exploration/sim_data/valid/occupied_volume_all_samples.pdf', scale=2)

plotly.offline.plot(fig, filename='./volume_exploration_vs_time.html', image='png')

'file:///home/mfaria/Flying_Octomap_code/src/_generate_plots/Exploration/volume_exploration_vs_time.html'

In [ ]:
def percentExplored(file_path):
    df = pd.read_csv(file_path)
    d = ((df.occupied).max() * 100) / maximum_occupied_volume 
    print "Explored {} percent of the explorable volume.".format(d)
    return d
percent_occupied = pd.DataFrame([percentExplored(f) for f in files ])

In [ ]:
def statistics(df):
    print "The average is {} +- {}".format(df.mean()[0], df.std()[0])
    print "The median is {} bounded by {} and {}".format(df.median()[0],  df.min()[0],  df.max()[0])
statistics(percent_occupied)

In [ ]:
def volumeVsOccupied(file_path):
    df = pd.read_csv(file_path)
    return df.tail(1)


occupied_final_df = pd.concat([volumeVsOccupied(f) for f in files])
occupied_final_df = occupied_final_df.sort_values(by='time_ellapsed_millis')

In [ ]:



trace = go.Scatter(
        x=occupied_final_df.time_ellapsed_millis/1000/60,
        y=occupied_final_df.occupied,
        name='run ' ,
        mode='markers'
    )

layout = go.Layout(
    title='Occupied Volume vs Mission time',
    titlefont=dict(
        family='Courier New, monospace',
        size=24,
    ),
    xaxis=dict(
        title='Time in minutes',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        dtick=10
    ),
    yaxis=dict(
        title='Occupied volume (m )',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)


fig=dict(data=[trace], layout=layout)
pio.write_image(fig, '/media/mfaria/Ganesha/20171219_backup_home_catec/Margarida/20190100_exploration/sim_data/valid/occupied_vs_time.pdf', scale=2)

plotly.offline.plot(fig, filename='./occupied_vs_time.html', image='png')